In [1]:
import glob
from pathlib import Path

train_path = 'paddy -doctor-diseases-split-main/train'
test_path  = 'paddy -doctor-diseases-split-main/test'

print('train images')
for filepath in glob.glob(train_path + '/*/'):
    files = glob.glob(filepath + '*')
    print(f"{len(files)} \t {Path(filepath).name}")
    
print('test images')
for filepath in glob.glob(test_path + '/*/'):
    files = glob.glob(filepath + '*')
    print(f"{len(files)} \t {Path(filepath).name}")

train images
648 	 bacterial_leaf_blight
505 	 bacterial_leaf_streak
450 	 bacterial_panicle_blight
506 	 black_stem_borer
2351 	 blast
1257 	 brown_spot
868 	 downy_mildew
2151 	 hispa
1095 	 leaf_roller
2405 	 normal
1951 	 tungro
1273 	 white_stem_borer
765 	 yellow_stem_borer
test images
648 	 bacterial_leaf_blight
505 	 bacterial_leaf_streak
450 	 bacterial_panicle_blight
506 	 black_stem_borer
2351 	 blast
1257 	 brown_spot
868 	 downy_mildew
2151 	 hispa
1095 	 leaf_roller
2405 	 normal
1951 	 tungro
1273 	 white_stem_borer
765 	 yellow_stem_borer


In [3]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import seaborn as sns
import cv2
import albumentations as A

from albumentations.core.composition import Compose
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Add, Activation
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten

from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array, array_to_img

ModuleNotFoundError: No module named 'sklearn'

In [6]:
pip show keras


Name: keras
Version: 3.8.0
Summary: Multi-backend Keras
Home-page: 
Author: 
Author-email: Keras team <keras-users@googlegroups.com>
License: Apache License 2.0
Location: c:\Users\ASUS\anaconda3\envs\CPUU\Lib\site-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, packaging, rich
Required-by: tensorflow-intel
Note: you may need to restart the kernel to use updated packages.


In [4]:
SEED = 123
EPOCHS = 10
lr = 1e-4
valid_split = 0.2
input_size = 224
batch_size = 32

n_classes = len(glob.glob(train_path + '/*/'))
print(n_classes)

13


In [5]:
generator = ImageDataGenerator(rescale=1 / 224,
                                  rotation_range=10,
                                  shear_range=0.25,
                                  zoom_range=0.1,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  validation_split=valid_split,
#                                   preprocessing_function=get_transforms_train
                                 )

train_datagen = generator.flow_from_directory(train_path,
                                              target_size=(input_size, input_size),
                                              batch_size=batch_size,
                                              subset='training',
                                              seed=SEED)

valid_datagen = generator.flow_from_directory(train_path,
                                              target_size=(input_size, input_size),
                                              batch_size=batch_size,
                                              subset='validation',
                                              seed=SEED)

test_generator = ImageDataGenerator(rescale=1.0/224).flow_from_directory(directory=test_path,
                                                                    target_size=(input_size, input_size),
                                                                    batch_size=batch_size,
                                                                    #classes=['.'],
                                                                    shuffle=False,
                                                                    seed=SEED
                                                                   )

print(train_datagen.class_indices)
print(train_datagen.samples, valid_datagen.samples, test_generator.samples, )

Found 12983 images belonging to 13 classes.
Found 3242 images belonging to 13 classes.
Found 16225 images belonging to 13 classes.
{'bacterial_leaf_blight': 0, 'bacterial_leaf_streak': 1, 'bacterial_panicle_blight': 2, 'black_stem_borer': 3, 'blast': 4, 'brown_spot': 5, 'downy_mildew': 6, 'hispa': 7, 'leaf_roller': 8, 'normal': 9, 'tungro': 10, 'white_stem_borer': 11, 'yellow_stem_borer': 12}
12983 3242 16225


In [6]:
def create_model(input_size, n_classes):
    back_bone = tf.keras.applications.Xception(weights='imagenet', input_shape=(input_size,input_size,3), include_top=False)
    back_bone.summary()
    tf.keras.utils.plot_model(back_bone, to_file='xception.png')
    input_layer = Input(shape=(input_size,input_size,3))
    x = back_bone(input_layer)
    x = GlobalAveragePooling2D()(x)
    output_layer = Dense(n_classes, activation='softmax')(x)
    
    initializer = tf.keras.initializers.HeUniform()
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    loss = tf.keras.losses.categorical_crossentropy    
    
    model = Model(input_layer,output_layer)
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=['accuracy'])
    return model

model = create_model(input_size, n_classes)
model.summary()



Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 111, 111, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 111, 111, 32)         0         ['block1_conv1_bn[0][

In [7]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=15,
                                              monitor='val_loss',
                                              restore_best_weights=True,
                                              verbose=1)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(patience=5,
                                                 monitor='val_loss',
                                                 factor=0.75,
                                                 verbose=1)

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='xception.weights.best.keras',
                                                monitor='val_loss',
                                                verbose=1,
                                                save_best_only=True)

In [8]:
history = model.fit(train_datagen,
                    validation_data=valid_datagen,
                    batch_size=batch_size,
                    epochs=EPOCHS,
                    callbacks=[early_stop,reduce_lr, checkpoint])

Epoch 1/10


406/406 [==============================] - ETA: 0s - loss: 1.1426 - accuracy: 0.6368
Epoch 1: val_loss improved from inf to 0.55124, saving model to xception.weights.best.keras
406/406 [==============================] - 1845s 5s/step - loss: 1.1426 - accuracy: 0.6368 - val_loss: 0.5512 - val_accuracy: 0.8171 - lr: 1.0000e-04
Epoch 2/10
406/406 [==============================] - ETA: 0s - loss: 0.3849 - accuracy: 0.8799
Epoch 2: val_loss improved from 0.55124 to 0.28731, saving model to xception.weights.best.keras
406/406 [==============================] - 1769s 4s/step - loss: 0.3849 - accuracy: 0.8799 - val_loss: 0.2873 - val_accuracy: 0.9127 - lr: 1.0000e-04
Epoch 3/10
406/406 [==============================] - ETA: 0s - loss: 0.2142 - accuracy: 0.9348
Epoch 3: val_loss improved from 0.28731 to 0.20847, saving model to xception.weights.best.keras
406/406 [==============================] - 1774s 4s/step - loss: 0.2142 - accuracy: 0.9348 - val_loss: 0.2085 - val_accuracy: 

In [11]:
model.save("XceptionNEW.keras")
